In [60]:
import tensorflow.keras
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, Input, GlobalMaxPool1D, Activation
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
import os 

In [2]:
filename = '../dataset_text/miditokens.txt'
with open(filename) as f:
    miditokens = f.readlines()
    
miditokens = [tokens.strip().split(' ') for tokens in miditokens]

In [3]:
tokenizer = Tokenizer() # token -> int
tokenizer.fit_on_texts(miditokens)

In [95]:
sample = miditokens[1]
#print(sample)
#print(tokenizer.texts_to_sequences([sample])[0]) # Example of token input -> int output
#print("Sõna sagedus kogu andmestikus", tokenizer.word_counts['end'])
#print("Sõna -> indeks teisendus", tokenizer.word_index['note:c4:v112'])
#print("Indeks -> sõna teisendus", tokenizer.index_word[55])
#print(len(tokenizer.word_index))
#print(tokenizer.index_word)
#print(tokenizer.word_index)

In [113]:
# Turn all tokens to ints
midiTokensAsInt = tokenizer.texts_to_sequences(miditokens)
#midiTokensAsInt = np.array(midiTokensAsInt)

TypeError: unsupported operand type(s) for -: 'list' and 'float'

In [ ]:
# GENERATOR VARIANT

In [78]:
## https://medium.com/analytics-vidhya/train-keras-model-with-large-dataset-batch-training-6b3099fdf366
#
#def batchGenerator(trainData, VOCAB_SIZE, SEQ_LEN=30):
#    i = 0
#    while True:
#        yield loadDataBatch(trainData, VOCAB_SIZE, SEQ_LEN, i)
#        i += 1
#    
#            
#def loadDataBatch(trainData, VOCAB_SIZE, SEQ_LEN, i):
#    X = []
#    y = []
#    
#    # https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5
#    
#    song = trainData[i]
#    for i in range(0, len(song) - SEQ_LEN, 1):
#        X.append(song[i:i + SEQ_LEN])
#        y.append(song[i + SEQ_LEN])
#            
#    input_len = len(X)
#    
#    # reshape the input into a format compatible with LSTM layers
#    X = np.reshape(X, (input_len, SEQ_LEN, 1))
#    
#    # normalize input
#    X = X / float(VOCAB_SIZE)
#    y = to_categorical(y, num_classes = VOCAB_SIZE)
#    
#    return (X, y)
#
#VOCAB_SIZE = len(tokenizer.word_index)
#SEQ_LEN = 20
#
#batchGen = batchGenerator(midiTokensAsInt, VOCAB_SIZE, SEQ_LEN)

In [115]:
# NON-GENERATOR VARIANT

X = []
y = []

VOCAB_SIZE = len(tokenizer.word_index)
SEQ_LEN = 20

# https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5

for song in midiTokensAsInt:
    for i in range(0, len(song) - SEQ_LEN, 1):
        X.append(song[i:i + SEQ_LEN])
        y.append(song[i + SEQ_LEN])

input_len = len(X)

# reshape the input into a format compatible with LSTM layers
X = np.reshape(X, (input_len, SEQ_LEN, 1))

# normalize input
X = X / float(VOCAB_SIZE)
y = to_categorical(y)

In [86]:
## GENERATOR VARIANT
#
#model = Sequential()
#model.add(LSTM(
#    256,
#    input_shape=(20, 1),
#    return_sequences=True
#))
#model.add(Dropout(0.3))
#model.add(LSTM(512, return_sequences=True))
#model.add(Dropout(0.3))
#model.add(LSTM(256))
#model.add(Dense(256))
#model.add(Dropout(0.3))
#model.add(Dense(VOCAB_SIZE))
#model.add(Activation('softmax'))
#model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
#
#filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"    
#checkpoint = ModelCheckpoint(
#    filepath, monitor='loss', 
#    verbose=0,        
#    save_best_only=True,        
#    mode='min'
#)    
#callbacks_list = [checkpoint]     
##model.fit(network_input, network_output, epochs=200, batch_size=64, callbacks=callbacks_list)
#
#model.fit(batchGen, 
#          epochs=200,
#          callbacks=callbacks_list,
#          verbose=1)

In [116]:
model = Sequential()
model.add(LSTM(
    256,
    input_shape=(X.shape[1], X.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(VOCAB_SIZE+1))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]     
model.fit(X, y, epochs=200, batch_size=64, callbacks=callbacks_list)

Epoch 1/200
   62/26062 [..............................] - ETA: 2:13:13 - loss: 5.8195

KeyboardInterrupt: 

In [ ]:
from music21 import *

def text2midi(tokens):
    s = stream.Stream()

    currentOffset = 0
    currentToken = 0

    for token in tokens:

        splitToken = token.split(":")

        if token.startswith("tempo"):
            s.append(tempo.MetronomeMark(number=float(splitToken[1])))

        if token.startswith("timesig"):
            s.append(meter.TimeSignature(splitToken[1]))

        if token.startswith("note") and not token.endswith("OFF"):
            noteDuration = 0
            noteName = splitToken[1]
            noteVelocity = int(splitToken[2][1:])

            for element in tokens[currentToken+1:]:
                splitToken2 = element.split(":")
                if (element.startswith("wait")):
                    noteDuration += float(splitToken2[1])
                if (element.startswith("note") and element.endswith("OFF")):
                    if (noteName == splitToken2[1]):
                        newNote = note.Note(nameWithOctave=splitToken[1],  
                               quarterLength=float(noteDuration))
                        newNote.volume.velocity = int(splitToken[2][1:])
                        s.insert(currentOffset, newNote)
                        break

        if token.startswith("wait"):
            currentOffset += float(splitToken[1]) 

        currentToken += 1

    return s

text2midi(sample).show("midi")